# Exploring GLM fits to individual neurons

The data below were fit using a GLM with two separate kernels aligned at different event times, across three different conditions.

Conditions:
1. Stimulus side (Left or Right)
2. Bias block (P(Left) = {0.2, 0.5, 0.8})
3. Stimulus contrast 'Low' = (0.0625, 0.125), 'High' = (0.25, 1.), 'Zero' = (0.)

Kernels:
1. Aligned at stimulus onset ('stimOn')
2. Aligned at feedback time ('feedback_t')

This results in 2 kernels per neurons at 30 different possible conditions. The tools below allow inspection of per-neuron fits across conditions where available. Some neurons do not fire enough spikes over the trials with a given condition to be worth fitting.

## Select a session fit to examine:

In [1]:
from ipywidgets import interact, Dropdown
import os
import pandas as pd
import numpy as np
from oneibl import one
%matplotlib inline

# Get names of subjects for which fits are available
subjects = [x for x in os.listdir('./fits/') if os.path.isdir(f'./fits/{x}/')]
dates = {sub: os.listdir(f'./fits/{sub}/') for sub in subjects}

sub_widget = Dropdown(options = subjects)
sess_widget = Dropdown()

# Define a function that updates the content of y based on what we select for x
def update(*args):
    sess_widget.options = dates[sub_widget.value]
sub_widget.observe(update)

currfit = {}
@interact(subj=sub_widget, fit_name=sess_widget)
def get_fit(subj, fit_name):
    currfit.update(np.load(f'./fits/{subj}/{fit_name}', allow_pickle=True))

interactive(children=(Dropdown(description='subj', options=('ZM_2240',), value='ZM_2240'), Dropdown(descriptio…

In [2]:
from fitplot_funs import get_fullfit_cells, plot_cellkerns
from export_data import trialinfo_to_df
print('Session : {subject} with UUID : {session_uuid}'.format(**currfit))
one = one.ONE()
spikes, clus = one.load(currfit['session_uuid'], dataset_types=['spikes.times', 'spikes.clusters'])
trialdf = trialinfo_to_df(currfit['session_uuid'])

df = currfit['fits']
# We need to get rid of the neutral bias zero contrast fits. These have too few trials and are never fit.
droprows = df.xs([0.5, 'Zero'], level=['bias', 'contr'], drop_level=False).index
df.drop(index=droprows, inplace=True)
fitcells = get_fullfit_cells(df)
cell_widg = Dropdown(options=fitcells)
pltfun = lambda cell: plot_cellkerns(cell, [df, trialdf, spikes, clus, currfit['kern_length'],
                                            currfit['glm_binsize']])
print(currfit['kern_length'], currfit['glm_binsize'])
interact(pltfun, cell=cell_widg)


Connected to https://alyx.internationalbrainlab.org as berk.gercek
Session : ZM_2240 with UUID : 510b1a50-825d-44ce-86f6-9678f5396e02
Connected to https://alyx.internationalbrainlab.org as berk.gercek
0.6 0.02


interactive(children=(Dropdown(description='cell', options=('cell10', 'cell100', 'cell102', 'cell103', 'cell10…

<function __main__.<lambda>(cell)>